In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import numpy as np
import math

In [2]:
def random_start_indices(start, end, num_samples):
    return np.random.randint(start, end, num_samples)

In [3]:
i_2 = random_start_indices(start=0, end=8170499, num_samples = 4000)
i_3 = random_start_indices(start=32558400,end=118958399, num_samples =4000)
i_4 = random_start_indices(start=0, end=35360700, num_samples=4000)
i_5 = random_start_indices(start=0, end=117653000, num_samples= 4000)
i_6 = random_start_indices(start=57993700,end=114153699, num_samples =4000)
i_7 = random_start_indices(start=0, end =123636300, num_samples= 4000)

In [4]:
def get_selected_rows(parquet_path, indices, csv_path, chunk_size):
    df = pd.read_parquet(parquet_path)
    rows= []
    for index in indices:
        for i in range(chunk_size):
            rows.append(df.loc[index+i*50])

    df = pd.DataFrame(rows)
    df.to_csv(csv_path)

In [15]:
get_selected_rows(parquet_path="./cleaned_parquet/007/Month 2/DD072024007_left wrist_101697_2024-09-18 11-15-01.parquet", indices=i_2, csv_path="month_2.csv", chunk_size=50)

In [16]:
get_selected_rows(parquet_path="cleaned_parquet/007/Month 3/DD072024007_left wrist_101696_2024-10-22 15-22-36.parquet", indices=i_3, csv_path="month_3.csv", chunk_size=50)

In [17]:
get_selected_rows(parquet_path="cleaned_parquet/007/Month 4/DD072024007_left wrist_101697_2024-11-20 13-12-43.parquet", indices=i_4, csv_path="month_4.csv", chunk_size=50)

In [5]:
get_selected_rows(parquet_path="cleaned_parquet/007/Month 5/DD072024007_left wrist_101699_2024-11-19 10-32-50.parquet", indices=i_5, csv_path="month_5.csv", chunk_size=50)

In [6]:
get_selected_rows(parquet_path="cleaned_parquet/007/Month 6/DD072024007_left wrist_101696_2025-01-30 13-12-54.parquet", indices=i_6, csv_path="month_6.csv", chunk_size=50)

In [7]:
get_selected_rows(parquet_path="cleaned_parquet/007/Month 7/DD072024007__100889_2025-03-26 12-05-58.parquet", indices=i_7, csv_path="month_7.csv", chunk_size=50)

In [5]:
def labeling_a_seq(seq_start_date, seq_end_date, df_labels):
    df_labels["start_date"] = pd.to_datetime(df_labels["start_date"])
    df_labels["end_date"] = pd.to_datetime(df_labels["end_date"])
    for i in range(len(df_labels["average"])):
        if df_labels["start_date"].iloc[i] <= seq_start_date and df_labels["end_date"].iloc[i] >= seq_end_date:
            
            return [df_labels["average"].iloc[i], df_labels["phq_9"].iloc[i], df_labels["cgis"].iloc[i], df_labels["gad_7"].iloc[i], df_labels["wsas"].iloc[i], df_labels["qids"].iloc[i]]

In [ ]:
def label_seqs(csv_file, loc): #this code is couples to the the csv_file as well as seq length.
    #change the number of rows of df_features you need and the seq length accordingly
    labels = []
    df_labels = pd.read_csv("cleaned_parquet/007/labels.csv")
    df_features = pd.read_csv(csv_file)
    df_features = df_features.head(180950)
    print(df_features)
    #every 50th row is the start of a new seq. If seq_len is then then replace 50 by 10 below
    for i in range(0, len(df_features["date"]), 50):
        seq_start_date = pd.to_datetime(df_features["date_only"].iloc[i])
        labels.append(labeling_a_seq(seq_start_date, seq_start_date, df_labels))
        print(i)
    print(len(labels))

    labels = np.array(labels)
    df_y = pd.DataFrame()
    df_y["average"] = labels[:, 0]
    df_y["phq_9"] = labels[:, 1]
    df_y["cgis"] = labels[:, 2]
    df_y["gad_7"] = labels[:, 3]
    df_y["wsas"] = labels[:, 4]
    df_y["qids"] = labels[:, 5]

    df_y.to_csv(loc)

In [6]:
df_labels = pd.read_csv("cleaned_parquet/007/labels.csv")
a_start_date= pd.to_datetime("2024-10-16")
a_end_date=pd.to_datetime("2024-10-17")

labeling_a_seq(a_start_date, a_end_date, df_labels)

['bad', 'bad', 'neutral', 'neutral', 'neutral', 'bad']

In [15]:
for i in range(2, 8):
    label_seqs(csv_file=f"cleaned_parquet/007/month_{i}.csv", loc=f"cleaned_parquet/007/month_{i}_labels.csv")

In [18]:
def label_manually(csv_path, l):
    df = pd.read_csv(csv_path)
    labels = np.full(15000,l)
    df["label"] = labels
    return df

In [19]:
df_3 = label_manually(csv_path="cleaned_parquet/007/month_3.csv", l = "neutral")
df_4 = label_manually(csv_path="cleaned_parquet/007/month_4.csv", l = "bad")
df_6 = label_manually(csv_path="cleaned_parquet/007/month_6.csv", l = "good")

df_all = pd.concat([df_3, df_4, df_6])
df_all.to_csv("cleaned_parquet/007/sample.csv")

In [16]:
'''
df_2 = pd.read_csv("cleaned_parquet/007/month_2.csv")
df_3 = pd.read_csv("cleaned_parquet/007/month_3.csv")
df_4 = pd.read_csv("cleaned_parquet/007/month_4.csv")
df_5 = pd.read_csv("cleaned_parquet/007/month_5.csv")
df_6 = pd.read_csv("cleaned_parquet/007/month_6.csv")
df_7 = pd.read_csv("cleaned_parquet/007/month_7.csv")
'''
df_l_2=pd.read_csv("cleaned_parquet/007/month_2_labels.csv")
df_l_3=pd.read_csv("cleaned_parquet/007/month_3_labels.csv")
df_l_4=pd.read_csv("cleaned_parquet/007/month_4_labels.csv")
df_l_5=pd.read_csv("cleaned_parquet/007/month_5_labels.csv")
df_l_6=pd.read_csv("cleaned_parquet/007/month_6_labels.csv")
df_l_7=pd.read_csv("cleaned_parquet/007/month_7_labels.csv")

In [17]:
#df_all = pd.concat([df_2, df_3, df_4, df_5, df_6, df_7])
#df_all.to_csv("cleaned_parquet/007/sample_features.csv")
df_labels_all = pd.concat([df_l_2, df_l_3, df_l_4, df_l_5, df_l_6, df_l_7])
df_labels_all.to_csv("cleaned_parquet/007/sample_labels.csv")